In [2]:
import pandas as pd
import numpy as np

import scipy
from scipy import stats
import statsmodels as sm
from statsmodels.regression.linear_model import OLS

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline


from datetime import datetime
import time
import datetime


import eikon as ek
creds = 'f5a341b614d945b9b93a71fe53348ba119d11e99'
#API Key Lionel: d505f265d9bd4c0cb7df44481ece31a0ad8d1974
#API Key Raphi: f5a341b614d945b9b93a71fe53348ba119d11e99
#API Key Laurin:
ek.set_app_key(creds)

In [32]:
#Define the Future Contracts

Futures = ['CLc1',
            'Cc1',
            'GCc1',
            'LCOc1',
            'LCc1',
            'NGLNMc1',
            'NGc1',
            'OJc1',
            'PAc1',
            'PLc1',
            'SIc1',
            'Wc1'
          ]

In [15]:
#Define the Makro Data

Macro_Data = ['CHCPIY=ECI',
               'CHJOB=ECI',
               'CHPMI=ECI',	
               'CNCPI=ECI',
               'CNPMIB=ECI',
               'CNURUA=ECI',
               'EUHICY=ECI',
               'EUPMI=ECI',
               'EUUNR=ECI',
               'RUCPIY=ECI',
               'RUPMIM=ECI',
               'RUUNR=ECI',
               'USCPI=ECI',
               'USPMI=ECI',
               'USUNR=ECI'
            ]

In [61]:
df, err = ek.get_data(
    instruments = ['CLc1'],
    fields = ['TR.CLOSEPRICE(Frq=CM,SDate=2012-01-01,EDate=2022-01-01)']
)

display(df)

,Instrument,Close Price
0,CLc1,98.28
1,CLc1,106.91
2,CLc1,102.93
3,CLc1,104.89
4,CLc1,86.5
...,...,...
115,CLc1,68.51
116,CLc1,75.12
117,CLc1,83.22
118,CLc1,67.0


In [58]:
df, err = ek.get_data(
    instruments = ['CLc1'],
    fields = ['TR.SETTLEMENTPRICE(Frq=CM,SDate=2012-01-01,EDate=2022-01-01)']
)

display(df)

,Instrument,Settlement Price
0,CLc1,98.48
1,CLc1,107.07
2,CLc1,103.02
3,CLc1,104.87
4,CLc1,86.53
...,...,...
115,CLc1,68.5
116,CLc1,75.03
117,CLc1,83.57
118,CLc1,66.18


In [56]:
df = ek.get_timeseries(Futures, 
                       fields = 'close',
                       interval='monthly', 
                       start_date='2012-01-01', 
                       end_date='2022-01-01',
                       )

display(df)

CLOSE,CLc1,Cc1,GCc1,LCOc1,LCc1,NGLNMc1,NGc1,OJc1,PAc1,PLc1,SIc1,Wc1
Date,,,,,,,,,,,,
2012-01-31,98.48,639.0,1737.8,110.98,124.85,56.25,2.503,210.0,685.95,1585.5,33.233,666.0
2012-02-29,107.07,656.5,1709.9,122.66,127.65,59.38,2.616,190.55,706.65,1691.1,34.583,664.25
2012-03-31,103.02,644.0,1669.3,122.88,120.45,61.5,2.126,164.5,653.15,1638.3,32.469,660.75
2012-04-30,104.87,660.25,1663.4,119.47,118.75,57.87,2.285,140.85,681.75,1567.7,30.959,647.75
2012-05-31,86.53,555.25,1562.6,101.87,117.1,53.98,2.422,112.05,612.1,1416.1,27.741,643.75
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,68.5,534.0,1815.0,72.99,118.95,127.71,4.377,134.55,2468.3,1013.9,23.96,706.75
2021-09-30,75.03,536.75,1755.3,78.52,120.575,251.18,5.867,132.5,1902.6,960.9,22.021,725.5
2021-10-31,83.57,568.25,1783.0,84.38,127.375,165.98,5.426,120.35,1986.1,1019.6,23.945,772.75


In [45]:
macro = ek.get_timeseries(Macro_Data,
                          fields='value',
                          interval='monthly',
                          start_date='2012-01-01',
                          end_date='2022-01-01',
                          )     

display(macro)

VALUE,CHCPIY=ECI,CHJOB=ECI,CHPMI=ECI,CNCPI=ECI,CNPMIB=ECI,CNURUA=ECI,EUHICY=ECI,EUPMI=ECI,EUUNR=ECI,RUCPIY=ECI,RUPMIM=ECI,RUUNR=ECI,USCPI=ECI,USPMI=ECI,USUNR=ECI
Date,,,,,,,,,,,,,,,
2012-01-31,-0.8,2.8,48.0,4.5,50.5,<NA>,2.7,<NA>,10.9,4.2,<NA>,6.3,0.3,54.2,8.3
2012-02-29,-0.9,2.8,48.2,3.2,51.0,<NA>,2.7,<NA>,11.1,3.7,<NA>,6.2,0.2,53.3,8.3
2012-03-31,-1.0,2.8,50.6,3.6,53.1,<NA>,2.7,<NA>,11.2,3.7,<NA>,6.3,0.2,53.5,8.2
2012-04-30,-1.0,2.8,46.5,3.4,53.3,<NA>,2.6,<NA>,11.3,3.6,<NA>,5.6,0.2,55.2,8.2
2012-05-31,-1.0,2.9,45.8,3.0,50.4,<NA>,2.4,<NA>,11.4,3.6,<NA>,5.2,-0.2,53.2,8.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,0.9,2.9,67.0,0.8,50.1,5.1,3.0,61.4,7.5,6.7,46.5,4.4,0.3,59.7,5.2
2021-09-30,0.9,2.8,67.6,0.7,49.6,4.9,3.4,58.6,7.3,7.4,49.8,4.3,0.4,60.5,4.7
2021-10-31,1.2,2.7,65.7,1.5,49.2,4.9,4.1,58.3,7.2,8.1,51.6,4.3,0.9,60.8,4.6
